In [2]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','amount':'{:,.2f}'
}
cols = 'trade name qty price active'.split()

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 5, 16), datetime.date(2022, 5, 15))

In [3]:
yesterday = date(2021, 5, 13)
yesterday

datetime.date(2021, 5, 13)

In [4]:
format_dict = {
    'qty':'{:,}',
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%',  
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',    
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
}

### Matching check with profits in portmy database with 2022-q1 data (22-05-14 )

In [5]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(57, 18)

In [6]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(33, 23)

In [7]:
df_merge = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge.shape

(57, 41)

In [8]:
names = df_merge[df_merge._merge == 'right_only']['name']
names

Series([], Name: name, dtype: object)

In [9]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

''

In [10]:
sql = """
SELECT id+9000 AS id, name, market, max_price, min_price, daily_volume AS volume, beta 
FROM stocks
WHERE name IN (%s)"""
sql = sql % in_p
my_stocks = pd.read_sql(sql, conmy)
my_stocks

,id,name,market,max_price,min_price,volume,beta


In [11]:
my_stocks.dtypes

id           object
name         object
market       object
max_price    object
min_price    object
volume       object
beta         object
dtype: object

In [12]:
my_stocks['status'] = 'O'
my_stocks['buy_target'] = 0.00
my_stocks['sell_target'] = 0.00
my_stocks['cost'] = 0.00
my_stocks['qty'] = 0
my_stocks['buy_spread'] = -4
my_stocks['sell_spread'] = 4
my_stocks['available_qty'] = 0
my_stocks['bl'] = 0
my_stocks['sh'] = 0
my_stocks['reason'] = 'XXX'
my_stocks.dtypes

id                object
name              object
market            object
max_price         object
min_price         object
volume            object
beta              object
status            object
buy_target       float64
sell_target      float64
cost             float64
qty                int64
buy_spread         int64
sell_spread        int64
available_qty      int64
bl                 int64
sh                 int64
reason            object
dtype: object

In [13]:
rearrange_col = 'id name max_price min_price status buy_target sell_target volume beta cost qty buy_spread sell_spread available_qty bl sh reason market'.split()
rearrange_col

['id',
 'name',
 'max_price',
 'min_price',
 'status',
 'buy_target',
 'sell_target',
 'volume',
 'beta',
 'cost',
 'qty',
 'buy_spread',
 'sell_spread',
 'available_qty',
 'bl',
 'sh',
 'reason',
 'market']

In [14]:
my_stocks = my_stocks[rearrange_col]

In [15]:
rcds = my_stocks.values.tolist()
len(rcds)

0

In [16]:
for rcd in rcds:
    print(rcd)

In [17]:
for rcd in rcds:
    conlite.execute("""INSERT INTO stocks (id, name, max_price, min_price, status, buy_target, sell_target, volume, beta, cost, qty, buy_spread, sell_spread, available_qty, bl, sh, \
reason, market) \
VALUES( ?, ?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?)""", rcd)

### End of Process insert profit records to stocks

### Process for stock records that are not in profits table

In [18]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(57, 18)

In [19]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(33, 23)

In [20]:
df_merge2 = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge2.shape

(57, 41)

In [21]:
names = df_merge2[df_merge2._merge == 'left_only']['name']
names

2        ASP
4        BGC
6        DCC
7        DIF
9      HREIT
10     JASIF
11     MAKRO
12       MCS
14       NER
16      SCCC
17      SENA
21       TMT
23     WHART
24       BCH
27    DOHOME
31       ORI
36       STA
37     SYNEX
45       KCE
48       PTT
49     PTTGC
50     RATCH
52       SCC
56        IP
Name: name, dtype: object

In [22]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASP', 'BGC', 'DCC', 'DIF', 'HREIT', 'JASIF', 'MAKRO', 'MCS', 'NER', 'SCCC', 'SENA', 'TMT', 'WHART', 'BCH', 'DOHOME', 'ORI', 'STA', 'SYNEX', 'KCE', 'PTT', 'PTTGC', 'RATCH', 'SCC', 'IP'"

In [23]:
sql = '''
SELECT *
FROM stocks
WHERE name IN (%s)
ORDER BY market, name'''
sql = sql % in_p
lite_stocks = pd.read_sql(sql, conlite)
lite_stocks[['market','name']]

,market,name
0,SET,ASP
1,SET,BGC
2,SET,DCC
3,SET,DIF
4,SET,HREIT
5,SET,JASIF
6,SET,MAKRO
7,SET,MCS
8,SET,NER
9,SET,SCCC


### End of Process to delete stock record that are not in profits and avalable_qty = 0

In [ ]:
sql = '''
SELECT *
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.dtypes

In [ ]:
dividend.sort_values(['xdate'],ascending=[True]).style.format(format_dict)

In [ ]:
df_merge = pd.merge(stocks, dividend, on='name', how='outer', indicator=True)
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]
#no_dividend.sort_values(['xdate'],ascending=[True])

### End of Process

### Consensus table

In [ ]:
sql = '''
SELECT * FROM consensus ORDER BY name'''
css = pd.read_sql(sql, conlite)
css

In [ ]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
df_prf = pd.read_sql(sql, conmy)
df_prf.shape

In [ ]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
df_stk = pd.read_sql(sql, conlite)
df_stk.shape

In [ ]:
df_merge = pd.merge(df_prf,df_stk,on='name',how='outer',indicator=True)
df_merge.shape

In [ ]:
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]

In [ ]:
right_mask= df_merge._merge == 'right_only'
df_merge[right_mask]

In [ ]:
both_mask= df_merge._merge == 'both'
df_merge[both_mask]

In [ ]:
sql = '''
SELECT * FROM SALES ORDER BY name'''
sales = pd.read_sql(sql, conlite)
sales.shape[0]

In [ ]:
sql = '''
DELETE FROM sales
WHERE name = "TCAP"'''
rp = conlite.execute(sql)
rp.rowcount